В этом ноутбуке мы будем анализировать распределение токенов и их комбинаций в словах.

Используя токенизатор и N-граммный анализ, мы исследуем повторяющиеся паттерны, что поможет в задачах стемминга и лемматизации.

In [23]:
import os

from tokenizers import Tokenizer

tokenizer_uni = Tokenizer.from_file('../data/processed/tokenizer/words_unigram_5000.tokenizer.json')

In [24]:
with open('../data/processed/word_freqs/freq_1000000_oshhamaho.txt') as f:
    words = f.read().split('\n')

In [25]:
token_dist_dir = '../data/processed/token_distribution'
os.makedirs(token_dist_dir, exist_ok=True)

In [26]:
from collections import defaultdict
import nltk
from tqdm import tqdm


def create_token_ng_distribution(words, n=5):
    token_ngrams_freq_dist = nltk.FreqDist()
    ngrams_tokens = defaultdict(list)

    pgbar = tqdm(sorted(words), desc=f'create {n}-gram distribution')
    for word in pgbar:
        tokens = tokenizer_uni.encode(word).tokens
        token_ids = [tokenizer_uni.token_to_id(token) for token in tokens]

        ngrams = tuple(nltk.ngrams(tokens, n=n))
        token_ngrams_freq_dist.update(ngrams)
        for ng in ngrams:
            ngrams_tokens[ng].append((tokens, token_ids))

    return token_ngrams_freq_dist, ngrams_tokens

In [27]:
import csv
import pandas as pd


def is_valid_stem(stem, freq, min_freq=10, max_freq=400, min_len=4):
    # малая частота стема говорит о том что она возможно слишком специфичная и длинная
    if freq < min_freq:
        return False

    # большая частота говорит о том что она возможно слишком общая и короткая
    if freq > max_freq:
        return False

    # если длина стема слишком мала, кандидат тоже не подходит
    if len(stem) < min_len:
        return False

    # todo можно проверить на вхождение в список префиксов и суффиксов, так точность должна увеличиться
    return True


def save_by_stem(df_data, n, ng_name, freq):
    df = pd.DataFrame(df_data)
    # сортируя по количеству токенов в слове, наверху будут слова слова близкие к корню, а внизу сложносоставные слова
    df = df.sort_values('word_ng_len', ascending=True)

    f_name = f'({freq}){ng_name}'
    f_path = f'{token_dist_dir}/{n}/{f_name}.csv'
    df.to_csv(f_path, index=False, sep=',', quoting=csv.QUOTE_MINIMAL, header=True)


def choose_stem(token_ngrams_freq_dist, ngrams_tokens, n=5, is_save_by_stem=False):
    if is_save_by_stem:
        os.makedirs(f'{token_dist_dir}/{n}', exist_ok=True)

    data = []
    freq_dist_sorted = sorted(token_ngrams_freq_dist.items(), key=lambda x: x[1], reverse=True)
    pgbar = tqdm(freq_dist_sorted, desc=f'choose {n}-gram stem')
    for ng, freq in pgbar:
        # фильтруя n-граммы по частоте, получим часть слова которая встречается в составе других слов
        # то есть часть слова которую можно использовать для поиска похожих слов (стемминг)

        stem = ''.join(ng)
        if not is_valid_stem(stem, freq):
            continue

        df_data = []
        for _tokens, _token_ids in ngrams_tokens[ng]:
            word = ''.join(_tokens)
            df_data.append({
                'stem_ng_len': n,
                'stem': stem,
                'word_ng_len': len(_tokens),
                'word': word,
                'template': word.replace(stem, '?' * len(stem)),
                'tokens': '|'.join(_tokens),
            })

        if is_save_by_stem:
            ng_name = '_'.join(ng)
            save_by_stem(df_data, n, ng_name, freq)

        data.extend(df_data)

    return data

In [28]:
data = []
for n in range(1, 7):
    token_fd, ngrams_tokens = create_token_ng_distribution(words, n=n)
    data_i = choose_stem(token_fd, ngrams_tokens, n=n, is_save_by_stem=False)
    data.extend(data_i)

choose 6-gram stem: 100%|██████████| 191758/191758 [00:00<00:00, 1951756.06it/s]


In [29]:
stem_df = pd.DataFrame(data)
# Если стема полностью повторяет слово, она выбрана вполне удачно.
# В некоторых случаях возможно что это даже лемма, ну или поиск леммы сильно упрощается

# здесь не обязательно сравнивать строки полностью, достаточно количество токенов
stem_df['stem_is_full_word'] = stem_df['stem_ng_len'] == stem_df['word_ng_len']

In [21]:
sort_values = ['stem_ng_len', 'stem', 'word_ng_len', 'word']
stem_df = stem_df.sort_values(sort_values, ascending=True)
stem_df.to_csv(f'{token_dist_dir}/stem_candidates.csv.gz', index=False, sep=',', quoting=csv.QUOTE_MINIMAL, header=True,
               compression='gzip')

In [31]:
import re
# stem_df set index to template

replace_many_q = lambda x: re.sub(r'\?{1,}', '*', x)

stem_df['template_short'] = stem_df['template'].apply(replace_many_q)
stem_df.set_index('template_short', inplace=True)
stem_df

,stem_ng_len,stem,word_ng_len,word,template,tokens,stem_is_full_word
template_short,,,,,,,
Ды*кIуэнури,1,зэрыте,5,ДызэрытекIуэнури,Ды??????кIуэнури,Ды|зэрыте|кIу|эну|ри,False
Ды*хьэжа,1,зэрыте,4,Дызэрытехьэжа,Ды??????хьэжа,Ды|зэрыте|хь|эжа,False
Къа*жам,1,зэрыте,3,Къазэрытежам,Къа??????жам,Къа|зэрыте|жам,False
Къа*кIуэIам,1,зэрыте,5,КъазэрытекIуэIам,Къа??????кIуэIам,Къа|зэрыте|кIу|э|Iам,False
Сы*лъэу,1,зэрыте,5,Сызэрытелъэу,Сы??????лъэу,Сы|зэрыте|лъ|э|у,False
...,...,...,...,...,...,...,...
*гъуэнщи,6,яфIэгъэщIэ,8,яфIэгъэщIэгъуэнщи,??????????гъуэнщи,я|фI|э|гъэ|щI|э|гъуэ|нщи,False
*гъуэныжкъым,6,яфIэгъэщIэ,10,яфIэгъэщIэгъуэныжкъым,??????????гъуэныжкъым,я|фI|э|гъэ|щI|э|гъуэн|ыж|къ|ым,False
*гъуэнын,6,яфIэгъэщIэ,8,яфIэгъэщIэгъуэнын,??????????гъуэнын,я|фI|э|гъэ|щI|э|гъуэн|ын,False


In [60]:
template_short_counts = pd.DataFrame(stem_df.value_counts('template_short'))

stem_df_with_counts = pd.merge(template_short_counts, stem_df, on='template_short')
stem_df_with_counts = stem_df_with_counts[(stem_df_with_counts[0] > 50) & (stem_df_with_counts[0] < 5000)]
stem_df_with_counts

,0,stem_ng_len,stem,word_ng_len,word,template,tokens,stem_is_full_word
template_short,,,,,,,,
*ри,4045,1,дэкI,2,дэкIри,????ри,дэкI|ри,False
*ри,4045,1,тэдж,2,тэджри,????ри,тэдж|ри,False
*ри,4045,1,гугъэ,2,гугъэри,?????ри,гугъэ|ри,False
*ри,4045,1,шхын,2,шхынри,????ри,шхын|ри,False
*ри,4045,1,жиIэ,2,жиIэри,????ри,жиIэ|ри,False
...,...,...,...,...,...,...,...,...
е*ым,51,4,лъытакъ,6,елъытакъым,е???????ым,е|лъ|ыт|а|къ|ым,False
е*ым,51,5,упщIакъ,7,еупщIакъым,е???????ым,е|у|п|щI|а|къ|ым,False
е*ым,51,5,хьэлIакъ,7,ехьэлIакъым,е????????ым,е|хь|э|лI|а|къ|ым,False


In [62]:
template_short_example_df = pd.DataFrame(
    stem_df_with_counts.groupby('template_short').apply(lambda x: x.sample(n=min(len(x), 10), random_state=1)).reset_index(drop=True)
)
template_short_example_df['template_short'] = template_short_example_df['template'].apply(replace_many_q)
template_short_example_df.to_csv(f'{token_dist_dir}/stem_template_examples.csv', index=False, sep=',', quoting=csv.QUOTE_MINIMAL,
                    header=True)
template_short_example_df

,0,stem_ng_len,stem,word_ng_len,word,template,tokens,stem_is_full_word,template_short
0,532,2,кхъухь,3,кхъухь-,??????-,кхъу|хь|-,False,*-
1,532,2,Исмэ,3,Исмэ-,????-,Ис|мэ|-,False,*-
2,532,3,жэуаплыныгъэ,4,жэуаплыныгъэ-,????????????-,жэуап|л|ыныгъэ|-,False,*-
3,532,4,къытхуи,5,къытхуи-,???????-,къ|ыт|ху|и|-,False,*-
4,532,2,дызэпсэ,3,дызэпсэ-,???????-,дызэ|псэ|-,False,*-
...,...,...,...,...,...,...,...,...,...
23535,173,3,элъти,5,ящIэлъти,ящI?????,я|щI|э|лъ|ти,False,ящI*
23536,173,3,ыхункIэ,5,ящIыхункIэ,ящI???????,я|щI|ы|ху|нкIэ,False,ящI*
23537,173,1,ыфынут,3,ящIыфынут,ящI??????,я|щI|ыфынут,False,ящI*
23538,173,3,ыркъэ,5,ящIыркъэ,ящI?????,я|щI|ыр|къ|э,False,ящI*


In [8]:
stem_df[stem_df['stem'] == 'джэгун'].head(n=30)

,stem_ng_len,stem,word_ng_len,word,template,tokens,stem_is_full_word
668461,2,джэгун,2,джэгун,??????,джэгу|н,True
668460,2,джэгун,3,Сыджэгун,Сы??????,Сы|джэгу|н,False
668462,2,джэгун,3,джэгунми,??????ми,джэгу|н|ми,False
668463,2,джэгун,3,джэгунри,??????ри,джэгу|н|ри,False
668464,2,джэгун,3,джэгунрэ,??????рэ,джэгу|н|рэ,False
668466,2,джэгун,3,дыджэгун,ды??????,ды|джэгу|н,False
668467,2,джэгун,3,зэрыджэгун,зэры??????,зэры|джэгу|н,False
668471,2,джэгун,3,уджэгун,у??????,у|джэгу|н,False
668474,2,джэгун,3,фызэрыджэгун,фызэры??????,фызэры|джэгу|н,False
668465,2,джэгун,4,джэгунрэт,??????рэт,джэгу|н|рэ|т,False


In [9]:
import re
from collections import Counter

templates_cnt = Counter(stem_df['template_short'].values)

In [10]:
templates_df = pd.DataFrame(templates_cnt.most_common(), columns=['template', 'freq'])
templates_df = templates_df[templates_df['freq'] > 10]

In [11]:
templates_df.to_csv(f'{token_dist_dir}/stem_templates.csv', index=False, sep=',', quoting=csv.QUOTE_MINIMAL,
                    header=True)

In [36]:
stem_df

,stem_ng_len,stem,word_ng_len,word,template,tokens,stem_is_full_word
140847,1,-Джэрий,2,Бат-Джэрий,Бат???????,Бат|-Джэрий,False
140863,1,-Джэрий,2,Кърым-Джэрий,Кърым???????,Кърым|-Джэрий,False
140885,1,-Джэрий,2,Хъан-Джэрий,Хъан???????,Хъан|-Джэрий,False
140891,1,-Джэрий,2,Хьэжы-Джэрий,Хьэжы???????,Хьэжы|-Джэрий,False
140842,1,-Джэрий,3,Адэл-Джэрий,Адэл???????,Адэ|л|-Джэрий,False
...,...,...,...,...,...,...,...
1600677,6,………………,16,…………………………………………,????????????…………,…|…|…|…|…|…|…|…|…|…|…|…|…|…|…|…,False
1600678,6,………………,16,…………………………………………,????????????…………,…|…|…|…|…|…|…|…|…|…|…|…|…|…|…|…,False
1600679,6,………………,16,…………………………………………,????????????…………,…|…|…|…|…|…|…|…|…|…|…|…|…|…|…|…,False
1600680,6,………………,16,…………………………………………,????????????…………,…|…|…|…|…|…|…|…|…|…|…|…|…|…|…|…,False


In [37]:
stem_df[['word', 'stem']].to_csv(f'{token_dist_dir}/stem_candidates.csv', index=False, sep=',',
                                 quoting=csv.QUOTE_MINIMAL, header=True)

In [46]:
stem_df[stem_df['stem_is_full_word'] == True].to_csv(f'{token_dist_dir}/stem_full_words.csv', index=False, sep=',',
                                                     quoting=csv.QUOTE_MINIMAL, header=True)